In [1]:
# Load in PyTorch's pretrained network
import torchvision.models as models
import torch
import numpy as np
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.nn.functional as func
import pandas as pd
from torch.utils.data.dataset import Dataset
from PIL import Image
from torch.utils.data.sampler import SubsetRandomSampler
from random import shuffle
import matplotlib.pyplot as plt
import os

In [2]:
class UtilFuncs():
    def plot_losses(self):
        return 0
    def plot_images(self):
        return 0

In [4]:
# Add separate transformations for testing and training 
# Augmentatiosn for data?
class CHXData(Dataset):
    def __init__(self, img_dir, path_to_labels):
        self.img_dir = img_dir
        self.data_df = pd.read_csv(path_to_labels)
        self.img_names = self.data_df['Image Index']
        self.labels = np.asarray(self.data_df.loc[:, self.data_df.columns != 'Image Index'])
        self.transform = transforms.Compose([
            transforms.Resize(512),
            # Should the normalization be from the statistics of the training set or the entire set?
            # Normalize by using images specific to this domain,
            transforms.ToTensor(),
            transforms.Normalize((0.50546, 0.50546, 0.50546), (0.2319, 0.2319, 0.2319))
        ])
        
    # Return size of the dataset
    def __len__(self):
        return len(self.data_df)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        # Converts to 3-Channel as Resnet takes in 3 channels
        # seems to repeat the same values for every channel
        image = self.transform(Image.open(img_path).convert('RGB'))
        label = self.labels[idx]
        return (image, label)

In [5]:
class CHXModel():
    def __init__(self):
        self.model = None
        self.losses = None
        self.optimizer = None
    
    # Freeze or un-freeze model layers as required
    def update_grad(self, grad_val):
        for param in self.model.parameters():
            param.requires_grad = grad_val
            
    # Initial model setup
    def set_up_model(self, n_classes, lr):
        self.model = models.resnet34(pretrained=True, progress=True)
        # Freeze all layers
        self.update_grad(False)
        # Resnet has one fully connected layer, which outputs dimensions of n_classes
        num_ftrs = self.model.fc.in_features
        self.model.fc = torch.nn.Linear(num_ftrs, n_classes)
        self.model.cuda()
        # Use a binary cross-entropy loss function; Applies sigmoid internally (generating probabilities)
        # On the probabilities, cross-entropy loss is computed
        # Because we are doing multilabel, this is better as the value outtputted (unlike softmax)
        # is independent of the other values (while in softmax, probabilities must add up to one)
        self.losses =  torch.nn.BCEWithLogitsLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
    
    def load_checkpoint(self, file_path):
        checkpoint = torch.load(file_path)
        self.model.load_state_dict(checkpoint['state'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        return checkpoint['epoch']

    def save_checkpoint(self, state, filename):
        torch.save(state, filename)
    
    def evaluate(self, testloader):
        correct = 0
        incorrect = 0
        # Don't store intermediate values which would be needed for backpropgation (memory saving)
        with torch.no_grad():
            # Go through each batch in the testloader
            for X, y in testloader:
                input_img = Variable(X.cuda(non_blocking=True))
                labels = Variable(y.float().cuda(non_blocking=True))
                self.optimizer.zero_grad()
                output = self.model(input_img)
                print(output.shape)
    
    # Add in any data augmentation while training?
    def train(self, epochs, trainloader, val_loader, checkpoint_path=None):
        # Empty cache before training
        torch.cuda.empty_cache()
        if checkpoint_path != None:
            current_epoch = self.load_checkpoint(checkpoint_path)
            epochs = epochs - current_epoch
            
        for e in range(epochs):
            # Get loss per epoch
            running_loss = 0
            for i, (X, y) in enumerate(trainloader):
                input_img = Variable(X.cuda(non_blocking=True))
                labels = Variable(y.float().cuda(non_blocking=True))
                self.optimizer.zero_grad()
                output = self.model(input_img)
                loss = self.losses(output, labels)
                loss.backward()
                self.optimizer.step()
                running_loss = running_loss + loss.item()
            # Find the loss for the current epoch
            loss = running_loss/len(trainloader)
            print(f"Epoch {e + 1} - Loss: {loss}")
            state = {
                'epoch': e + 1,
                'state': self.model.state_dict(),
                'optimizer': self.optimizer.state_dict()
            }
            checkpoint_path = os.path.join('./checkpoints/checkpoint_epoch_' + str(e + 1) + '.pth.tar') 
            self.save_checkpoint(state, checkpoint_path)
            #self.evaluate(val_loader)

In [6]:
n_classes = 15
learning_rate = 1e-4
chx_model = CHXModel()
chx_model.set_up_model(n_classes, learning_rate)

In [7]:
path_to_train_data = './data/Train/'
path_to_test_data = './data/Test/'
path_to_val_data = './data/Val'
path_to_test_labels = './data/Labels/Test_Labels.csv'
path_to_train_labels = './data/Labels/Train_Labels.csv'
path_to_val_labels='./data/Labels/Val_Labels.csv'

In [8]:
# Create dataset and data loader
test_dataset = CHXData(path_to_test_data, path_to_test_labels)
train_dataset = CHXData(path_to_train_data, path_to_train_labels)
val_dataset =  CHXData(path_to_val_data, path_to_val_labels)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=100, shuffle=True)

In [ ]:
# Sample data
it = iter(test_loader)
for i in range(10):
    # Get a random index value for an index in the batch
    idx = np.random.randint(0, 99, 1)[0]
    # Use the random index to get an image, label in the batch
    image, label = [x[idx] for x in next(it)]
    plt.figure(num=None, figsize=(8, 6))
    plt.imshow(image.numpy().reshape(512, 512));
    plt.show()

In [10]:
# Train with the layers frozen for 5 epochs
chx_model.train(5, train_loader, val_loader)

Epoch 1 - Loss: 0.2166028259577892


FileNotFoundError: [Errno 2] No such file or directory: './data/checkpoints/checkpoint_epoch_1.pth.tar'

In [17]:
# NEED TO ADD DATA AUGMENTATION WHEN TRAINING
# NEED TO DEAL WITH IMBALANCE DATA
checkpoint_path = os.path.join('./checkpoints/checkpoint_epoch_' + str(0 + 1) + '.pth.tar')
state = {'epoch': 0 + 1, 'state': chx_model.model.state_dict(), 'optimizer': chx_model.optimizer.state_dict()}
chx_model.save_checkpoint(state, checkpoint_path)